In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
imgPaths = glob('./Selected/*.jpeg')

명함 이미지들에서 텍스트를 추출하고, 신뢰도(conf)가 높은 텍스트만 골라서 하나의 데이터프레임으로 모으는 과정

In [4]:
allBusinessCard = pd.DataFrame(columns=['id','text'])

for imgPath in  tqdm(imgPaths,desc='BusinessCard'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    # extract data and text 
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(float).astype(int)

    useFulData = df.query('conf >= 30')

    # Dataframe
    businessCard = pd.DataFrame()
    businessCard['text'] = useFulData['text']
    businessCard['id'] = filename
    
    # concatenation
    allBusinessCard = pd.concat((allBusinessCard,businessCard))

BusinessCard: 100%|██████████| 293/293 [03:59<00:00,  1.22it/s]


In [15]:
allBusinessCard.to_csv('businessCard.csv',index=False)